<a href="https://colab.research.google.com/github/fernandomaura97/Autonomous-HW/blob/main/Copia_de_belief_propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pgmpy

# Belief Propagation

In this lab session you will implement the belief propagation (BP) algorithm on a factor graph using the [pgmpy library](http://www.pgmpy.org). By the end of this session you will be able to
- **Create a factor graph** respresenting a **hidden Markov model**.
- **Answer inference queries** using the built in methods of **pgmpy**.
- **Experiment with** the fundamental concepts behind **probabilistic inference and message passing**.

## The Model

<img src="https://raw.githubusercontent.com/guillermoim/resources/main/makov_weather.png" width=750 height=200>

We will consider our previous Markov chain model of wheather change over time. For simplicy, we will assume that the **weather is stationary** over the seven days of a week, **and that it can take three different values:** `sunny`, `cloudy` and `rainy`. The probability of rain or clouds after a rainy day is $0.25$ and $0.5$, respectively. A sunny day is also followed by a sunny day or a cloudy day with probabilities $0.7$ and $0.25$, respectively. Finally, if a day is cloudy, the next day will also be cloudy with probability $0.35$ or it will rain with probability $0.4$.

<style type="text/css">@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style><div class="tg-wrap"><table style="border-collapse:collapse;border-spacing:0" class="tg"><thead><tr><th style="border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal" colspan="2" rowspan="2"></th><th style="border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal" colspan="3">Today's weather</th></tr><tr><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">sunny</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">cloudy</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">rainy</th></tr></thead><tbody><tr><td style="border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:middle;word-break:normal" rowspan="3">Yesterday's weather</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">sunny</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.7</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.25</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.05</td></tr><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">cloudy</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.25</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.35</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.4</td></tr><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">rainy</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.25</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.5</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.25</td></tr></tbody></table></div>

Let's assume that by some unfortunate reason we are not able to see the weather directly, but we can observe if someone makes use of an umbrella. This will be our observed random variable, which directly depends on the actual weather. The probability of carrying an umbrella is $0.95$ for a rainy day, $0.6$ for a cloudy day, and $0.2$ for a sunny day.  

<style type="text/css">@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style><div class="tg-wrap"><table style="border-collapse:collapse;border-spacing:0" class="tg"><thead><tr><th style="border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal" colspan="2" rowspan="2"></th><th style="border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal" colspan="2">Umbrella</th></tr><tr><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">true</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">false</th></tr></thead><tbody><tr><td style="border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:middle;word-break:normal" rowspan="3">Weather</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">sunny</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.2</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.8</td></tr><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">cloudy</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.6</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.4</td></tr><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">rainy</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.95</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-style:italic;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">.05</td></tr></tbody></table></div>


Finally, we know that the first day of the week is sunny, cloudy, or rainy with probabilities $0.3$, $0.4$ and $0.3$, respectively. 

## Creating the Factor graph

The model is composed of $14$ random variables, $7$ corresponding to the hidden weather state, and $7$ corresponding to the binary observed variable. Let's create the factors and variables enconding the transition and observational model.

In [ ]:
from pgmpy.factors.discrete import DiscreteFactor

weather = ["w1", "w2", "w3", "w4", "w5", "w6", "w7"]
umbrella = ["u1", "u2", "u3", "u4", "u5", "u6", "u7"]
variables = weather + umbrella
state_names = dict([(k, ["sunny", "cloudy", "rainy"]) for k in weather] +
                   [(k, [True, False]) for k in umbrella])

##we create a dictionary giving possible values of S,c,r to w1:w7, True or False to u1:u7
print(state_names["u3"],"\n")

for i in state_names:
  print(i,state_names[i])


[True, False] 

w1 ['sunny', 'cloudy', 'rainy']
w2 ['sunny', 'cloudy', 'rainy']
w3 ['sunny', 'cloudy', 'rainy']
w4 ['sunny', 'cloudy', 'rainy']
w5 ['sunny', 'cloudy', 'rainy']
w6 ['sunny', 'cloudy', 'rainy']
w7 ['sunny', 'cloudy', 'rainy']
u1 [True, False]
u2 [True, False]
u3 [True, False]
u4 [True, False]
u5 [True, False]
u6 [True, False]
u7 [True, False]


In [ ]:
len(state_names['w1'])

3

In [ ]:
from pgmpy.factors.discrete import DiscreteFactor

weather = ["w1", "w2", "w3", "w4", "w5", "w6", "w7"]
umbrella = ["u1", "u2", "u3", "u4", "u5", "u6", "u7"]
variables = weather + umbrella
state_names = dict([(k, ["sunny", "cloudy", "rainy"]) for k in weather] +
                   [(k, [True, False]) for k in umbrella])

def day_transition(d1, d2): # P(d2|d1)
     return DiscreteFactor(variables = [d1,d2],
                           cardinality = [3,3], ##given transition function from before
                           values = [0.7, 0.25, 0.05, 0.25, 0.35, 0.4, 0.25, 0.5, 0.25],
                           state_names = state_names                          
                           )
    
def take_umbrella_transition(d, u): # P(u|d)
    return DiscreteFactor( variables = [d,u],
                          cardinality = [3,2],
                          values = [0.2, 0.8, 0.6, 0.4, 0.95, 0.05],
                          state_names = state_names)

p = dict()
p["w1"] =  DiscreteFactor(variables = ["w1"],
                          cardinality = [3],
                          values = [0.3, 0.4, 0.3],
                          state_names = state_names)

p["w2|w1"] = day_transition("w1", "w2")
p['w3|w2'] = day_transition('w2','w3')
p['w4|w3'] = day_transition('w3','w4')
p['w5|w4'] = day_transition('w4','w5')
p['w6|w5'] = day_transition('w5','w6')
p['w7|w6'] = day_transition('w6','w7')

p["u1|w1"] = take_umbrella_transition("w1", "u1")
p['u2|w2'] = take_umbrella_transition("w2", "u2")
p['u3|w3'] = take_umbrella_transition("w3", "u3")
p['u4|w4'] = take_umbrella_transition("w4", "u4")
p['u5|w5'] = take_umbrella_transition("w5", "u5")
p['u6|w6'] = take_umbrella_transition("w6", "u6")
p['u7|w7'] = take_umbrella_transition("w7", "u7")




Now, let's build the model using the ```FactorGraph``` class. To do so, we first need to add all variable nodes with ```add_nodes_from```. Then, we add each factor ```f``` to the graph with```add_factors(f)``` and, for each of its variables ```v``` (available in ```f.variables```), we create an edge between ```f``` and ```v```  with ```add_edge(f,v)```. Finally, we verify that the factor graph is correct with the ```check_model``` method.

In [ ]:
from pgmpy.models import FactorGraph

G = FactorGraph()

assert set(variables) == set([v for f in p.values() for v in f.variables])

for i in p.values():
  print(i.variables)
  G.add_nodes_from(i.variables)
  G.add_factors(i)
  for j in i.variables:
    G.add_edge(i,j)

### ADD NODES AND EDGES TO THE FACTOR GRAPH

print(G)

print("Model is ok: ", G.check_model())                                                                                                          

['w1']
['w1', 'w2']
['w2', 'w3']
['w3', 'w4']
['w4', 'w5']
['w5', 'w6']
['w6', 'w7']
['w1', 'u1']
['w2', 'u2']
['w3', 'u3']
['w4', 'u4']
['w5', 'u5']
['w6', 'u6']
['w7', 'u7']
FactorGraph with 28 nodes and 27 edges
Model is ok:  True


In [ ]:
# import networkx as nx
# import pylab as plt
# nx.draw(G, with_labels=True)
# plt.show()

## Running Inference Queries

We will use now some methods implemented in `pgmpy` that perform exact inference.
Consider the following queries:

1. We observe that on **Thursday and Friday the umbrella is being used**. What is the **weather prediction for Saturday**?
2. What is the **weather prediction for Sunday**, with the **same observations**?
3. What is the **weather prediction for Sunday** if, instead, we observe that the **umbrella is *NOT* being used on Thursday and Friday**? 
4. What is the **most likely weather sequence for Monday and Tuesday**, if we observed that the **umbrella has been used both days**?


Answer the three previous queries using the `query` method of the `BeliefPropagation` class:
```python
    def query(self, variables, evidence=None, joint=True, show_progress=True):
        """
        Query method using belief propagation.

        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        joint: boolean
            If True, returns a Joint Distribution over `variables`.
            If False, returns a dict of distributions over each of the `variables`
```
You will need to normalize the output.

In [ ]:
from pgmpy.inference import BeliefPropagation
import numpy as np

bp = BeliefPropagation(G)

variables = ["w6"]
evidence = {"u4": True, "u5": True }
# MAKE QUERIES

q1 = bp.query(variables, evidence )
print("Q1:\n",q1)

variables2 = ["w7"]
evidence2 = {"u4": True, "u5": True }
q2 = bp.query(variables2, evidence2 )
print("Q2:\n",q2)

variables3 = ["w7"]
evidence3 = {"u4": False, "u5": False}
q3 = bp.query(variables3, evidence3)
print("Q3:\n",q3)

variables4 = ["w1" ,"w2"]
evidence4 = {"u1": True, "u2": True}
q4 = bp.query(variables4, evidence4)
print("Q4 :\n", q4)

amax = np.argmax(q4.values)
print("Maximum probability:", q4.values.flatten()[amax])
print("State assignment:", sorted(q4.assignment([amax])[0])) 

print("The most probable sequence is the one with maximum a posteriori probability,\nin this case W1(cloudy), W2(rainy)")


Q1:
 +------------+-----------+
| w6         |   phi(w6) |
+============+===========+
| w6(sunny)  |    0.3025 |
+------------+-----------+
| w6(cloudy) |    0.4080 |
+------------+-----------+
| w6(rainy)  |    0.2895 |
+------------+-----------+
Q2:
 +------------+-----------+
| w7         |   phi(w7) |
+============+===========+
| w7(sunny)  |    0.3861 |
+------------+-----------+
| w7(cloudy) |    0.3632 |
+------------+-----------+
| w7(rainy)  |    0.2507 |
+------------+-----------+
Q3:
 +------------+-----------+
| w7         |   phi(w7) |
+============+===========+
| w7(sunny)  |    0.5225 |
+------------+-----------+
| w7(cloudy) |    0.3077 |
+------------+-----------+
| w7(rainy)  |    0.1698 |
+------------+-----------+
Q4 :
 +------------+------------+--------------+
| w1         | w2         |   phi(w1,w2) |
+============+============+==============+
| w1(sunny)  | w2(sunny)  |       0.0246 |
+------------+------------+--------------+
| w1(sunny)  | w2(cloudy) |       0

## Implementing Belief Propagation

Implement your own Belief Propagation algorithm by filling in the provided code skeleton. You will need to provide the code to:
* Add the evidence factors
* Initialize a factor-to-variable message `f->v` and a variable-to-factor message `v->f` for each edge in the graph
* Update messages of each type
* Perform message passing using the collect_evidence and distribute_evidence algorithms
* Compute the marginal using the messages

See function comments for more details. Each comment line ```#IMPLEMENT``` is to be replaced with as many lines of code as you need. Method definitions should remain unchanged.

In [ ]:
# from functools import reduce
# import operator
# from collections import defaultdict
# from copy import deepcopy

# ##TEST
# class MyBeliefPropagation1:
#     def __init__(self, factor_graph):
#         assert factor_graph.check_model()
#         self.original_graph = factor_graph
#         self.variables = factor_graph.get_variable_nodes()
        
#         self.state_names = dict()
#         for f in self.original_graph.factors:
#             self.state_names.update(f.state_names)
#         self.bp_done = False
  
#     def initialize_messages1(self):
#       self.messages = defaultdict(dict)
#       print(self.messages)
#       c=0
#       for f in self.original_graph.get_factors():
#           c+=1
#           print("\n\n*********************KEYYYYYS********************\n")
#           print(f.variables)
#       # ret
#             # mes_f-v[v][f] = 
#             # mes_v-f[f][v] = 
#             #   # IMPLEMENT


# #test
# my_bp = MyBeliefPropagation1(G)
# j = my_bp.initialize_messages1()


In [ ]:
# print(G.get_factors())

# def factor_ones(self, v):
#         """
#         Returns a DiscreteFactor for variable v with all ones.
#         """
#         return DiscreteFactor([v], cardinality = [len(self.state_names[v])], values = [np.ones(len(self.state_names[v]))])
               



# for f in G.get_factors():
#   for v in f.variables:
#               print(v," ",f)
#               # v -> f  
      
#               print(v_f)
#               self.messages[v][f] = v_f
#               # f -> v
#               f_v = DiscreteFactor(f.variables, f.cardinality, ([1] * np.prod(f.cardinality)), state_names=self.state_names)
#               print(f_v)
#               self.messages[f][v] = f_v

In [44]:
from functools import reduce
import operator
from collections import defaultdict
from copy import deepcopy
import numpy as np


def prod(iterable):
    
    """
      Returns the product of all the items in the iterable given in as input.
    """

    return reduce(operator.mul, iterable, 1)

class MyBeliefPropagation:
    def __init__(self, factor_graph):
        assert factor_graph.check_model()
        self.original_graph = factor_graph
        self.variables = factor_graph.get_variable_nodes()
        
        self.state_names = dict()
        for f in self.original_graph.factors:
            self.state_names.update(f.state_names)
        self.bp_done = False

    def factor_ones(self, v):
        """
        Returns a DiscreteFactor for variable v with all ones.
        """
        return DiscreteFactor([v], cardinality = [len(self.state_names[v])], values = [np.ones(len(self.state_names[v]))])
               
    def initialize_messages(self):
        """
        This function creates, for each edge factor-variable, two messages: m(f->v) and 
        m(v->f). It initializes each message as a DiscreteFactor with all ones. It stores all
        the messages in a dict of dict. Keys of both dicts are either factors or variables.
        Messages are indexed as messages[to][from]. For example, m(x->y) is in messages[y][x].
        It's done this way because it will be useful to get all messages that go to a variable
        or a factor.
        """
        self.messages = defaultdict(dict)
        for f in self.working_graph.get_factors():        
          for v in f.variables:
            print("F:", f, "V:", v)
            mes_fv = self.factor_ones(v)
            mes_vf = self.factor_ones(v)
            print("MESSAGE F->V:", mes_fv, "\n\nMESSAGE V->F:", mes_vf)
            self.messages[v][f] = mes_fv
            self.messages[f][v] = mes_vf

    def factor_to_variable(self, f, v):
        """
        Computes message m from factor to variable. It computes it from all messages from all
        other variables to the factor (i.e. all variables connected the factor except v).
        Returns message m.
        """
        assert v in self.variables and f in self.working_graph.factors
        # IMPLEMENT
        print("DBG_FTOVAR")

        m_fv = self.messages[v][f].copy()
        connected_vars = set(f.variables) & (set(self.variables) - {v})
        if not connected_vars:
          m_vf = DiscreteFactor(f.variables, f.cardinality, ([1] * np.prod(f.cardinality)), state_names=state_names)
          m_fv = m_fv * m_vf
          
        for var in connected_vars:
          if var != v:
            m_vf = self.messages[f][var].copy()
            m_fv = m_vf * m_fv  
        m_fv.normalize(inplace = False)
        return m_fv          

    def variable_to_factor(self, v, f):
        """
        Computes message m from variable to factor. It computes it from all messages from all
        other factors to the variable (i.e. all factors connected the variable except f).
        Returns message m.
        """
        assert v in self.variables and f in self.working_graph.factors
        # IMPLEMENT

        connected_factors = []
        for factor in self.working_graph.get_factors():
          if v in factor.variables:
            if factor!= f: 
              print("BEFORE APPEND:\n", connected_factors)
              connected_factors.append(factor) ##use all connected factors to var i except for f
              print("AFTER APPEND:\n", connected_factors,"\n" )

        if connected_factors == []: 
          messages = [DiscreteFactor(f.variables, f.cardinality, ([1] * np.prod(f.cardinality)), state_names = state_names)]
        else:
          for factor in connected_factors:
            messages = [self.messages[v][factor]]
            print("MESSAGES AFTER [DEBUG VAR TO FACTOR]:", messages)

        m = prod(messages) ##message from var i to factor a is multiplicatory of messages from factors connected to var i
        m.normalize() # always
        return m 

    def get_evidence_factors(self, evidence):
        """
        For each evidence variable v, create a factor with p(v=e)=1. Recieves a dict of
        evidence, where keys are variables and values are variable states. Returns a list of
        DiscreteFactor.
        """
        # IMPLEMENT
        evidence_factors = []
        for var, s in evidence.items():
          cardinality = [len(self.state_names[var])]
          values = np.zeros((np.prod(cardinality)))
          state = list(self.state_names[var]).index(s)
          values[state] = 1
          evidence_factors.append(DiscreteFactor([var],cardinality,values, state_names = self.state_names))
        return evidence_factors

    def update(self, m_to, m_from):
        """
        Performs an update of a message depending on whether it is variable-to-factor or
        factor-to-variable.
        """
        # IMPLEMENT
        if m_from in self.variables: ##if from variable
          ## var i to factor a 
          self.messages[m_to][m_from] = self.variable_to_factor(m_from, m_to)
        else: ##from factor a to variable i
          self.messages[m_to][m_from] = self.factor_to_variable(m_from, m_to)
    
    def collect_evidence(self, node, parent=None):
        """
        Passes messages from the leaves to the root of the tree.
        The parent argument is used to avoid an infinite recursion.
        """
        for child in self.working_graph.neighbors(node):
          # IMPLEMENT
          print("")
          if child!= parent:
            self.collect_evidence(child,node)
            self.messages[node][child] = self.update(child,node)


    def distribute_evidence(self, node, parent=None):
        """
        Passes messages from the root to the leaves of the tree.
        The parent argument is used to avoid an infinite recursion.
        """
        for child in self.working_graph.neighbors(node):
          print("")
          if child!=parent:
            self.messages[child][node] = self.update(node,child)
            #self.distribute_evidence(child,node) ##Recursively
            # IMPLEMENT
    
    def set_evidence(self, evidence):
        """
        Generates a new graph with the evidence factors
        evidence (keys: variables, values: states)
        """
        evidence_factors = self.get_evidence_factors(evidence)
        self.working_graph = self.original_graph.copy()
        for f in evidence_factors:
            self.working_graph.add_factors(f)
            for v in f.variables:
              self.working_graph.add_edge(f,v)

        self.bp_done = False
    
    def run_bp(self, root):
        """
        After initializing the messages, this function performs Belief Propagation
        using collect_evidence and distribute_evidence from the given root node.
        """
        assert root in self.variables, "Variable not in the model"
        # IMPLEMENT
        self.initialize_messages()
        self.collect_evidence(root)
        self.distribute_evidence(root)
        self.bp_done = True

    def get_marginal(self, variable):
        """
        To be used after run_bp. Returns p(variable | evidence) unnormalized.
        """
        assert self.bp_done, "First run BP!"
        # IMPLEMENT

        node = self.working_graph.get_node_by_name(variable)
        factors = []
        for factor in self.working_graph.factors:
          if variable in factor.variables:
            factors.append(factor)
        
        joint_factor = prod(factors)
        marginal_factor = joint_factor.marginalize([v for v in joint_factor.variables if v != variable])
        return marginal_factor


Check that your implementation produces the same results for the first three queries P(w7|u4=t,u5=t), P(w7|u4=t,u5=t), and P(w6|u4=f,u5=f) as the ones given by the `BeliefPropagation` class. Do you need to run BP each time? Justify your answer.

In [47]:
my_bp = MyBeliefPropagation(G)
# evidence = 
# my_bp.set_evidence()
# my_bp.run_bp('u1')

# variables2 = ["w7"]
evidence2 = {"u4": True, "u5": True }

my_bp.set_evidence(evidence2)
my_bp.run_bp("u7")


F: +------------+-----------+
| w1         |   phi(w1) |
+============+===========+
| w1(sunny)  |    0.3000 |
+------------+-----------+
| w1(cloudy) |    0.4000 |
+------------+-----------+
| w1(rainy)  |    0.3000 |
+------------+-----------+ V: w1
MESSAGE F->V: +-------+-----------+
| w1    |   phi(w1) |
+=======+===========+
| w1(0) |    1.0000 |
+-------+-----------+
| w1(1) |    1.0000 |
+-------+-----------+
| w1(2) |    1.0000 |
+-------+-----------+ 

MESSAGE V->F: +-------+-----------+
| w1    |   phi(w1) |
+=======+===========+
| w1(0) |    1.0000 |
+-------+-----------+
| w1(1) |    1.0000 |
+-------+-----------+
| w1(2) |    1.0000 |
+-------+-----------+
F: +------------+------------+--------------+
| w1         | w2         |   phi(w1,w2) |
+============+============+==============+
| w1(sunny)  | w2(sunny)  |       0.7000 |
+------------+------------+--------------+
| w1(sunny)  | w2(cloudy) |       0.2500 |
+------------+------------+--------------+
| w1(sunny)  | w2(

UnboundLocalError: ignored

Use BP to compute the fourth query P(w1,w2|u1=t,u2=t). Note that we cannot do it with the ```get_marginal``` method as it is now. Define a method ```get_marginal_subset``` that receives a list of variables as input and, if they share a common factor, returns the marginal of the subset. Otherwise, throw an error. Check your result with the one produced by the `BeliefPropagation` class.